<a href="https://colab.research.google.com/github/fanny/information-retrieval/blob/master/lab3/tokenizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re

import pandas as pd
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('https://raw.githubusercontent.com/fanny/information-retrieval/master/lab2/data/results.csv')
data.head(10)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


,_id,author,date,section,sub_title,text,title,url
0,7,NaN,05/04/2019 |,agronegocio,NaN,NaN,\t\t\t\t,https://www.gazetadopovo.com.br/agronegocio/ex...
1,2,NaN,NaN,wiseup-news,NaN,NaN,NaN,https://www.gazetadopovo.com.br/wiseup-news/bo...
2,3,NaN,NaN,futebol,NaN,NaN,NaN,https://especiais.gazetadopovo.com.br/futebol/...
3,9,NaN,05/04/2019 |,agronegocio,NaN,NaN,Suspensão de frigoríficos não impediu alta de ...,https://www.gazetadopovo.com.br/agronegocio/ag...
4,1,The Daily Signal,NaN,ideias,NaN,Pessoas que buscam uma identidade transsexual ...,Populares,https://www.gazetadopovo.com.br/ideias/nao-se-...
5,5,Gazeta do Povo,NaN,curitiba,NaN,"Depois de uma semana de calor, Curitiba terá u...",Populares,https://www.gazetadopovo.com.br/curitiba/previ...
6,40,"Flávia Alves, especial para a Gazeta do Povo",NaN,viver-bem,NaN,As promessas são muitas: combater o envelhecim...,NaN,https://www.gazetadopovo.com.br/viver-bem/saud...
7,6,Agência Estado,NaN,republica,NaN,"O líder do MDB na Câmara, Baleia Rossi (SP), p...",Populares,https://www.gazetadopovo.com.br/republica/pec-...
8,20,Daniel Malucelli,[05/04/2019],esportes,mercado,O meia-atacante Maicosuel está com a situação ...,Paraná não descarta dispensa de Maicosuel ante...,https://www.gazetadopovo.com.br/esportes/paran...
9,10,Renyere Trovão,NaN,automoveis,NaN,Concretizar a compra de um carro novo nem semp...,Populares,https://www.gazetadopovo.com.br/automoveis/car...


In [0]:
clean_text = data['text'].dropna()
def filter_by_criteria(text, criteria):
  filtered_words = re.findall(criteria, text)
  filtered_words = ' '.join(filtered_words)
 
  return filtered_words

In [0]:
def get_words_filtered(criteria):
  filtered_text = clean_text.apply(lambda text: filter_by_criteria(text, criteria))
  all_filtered_text = filtered_text.str.cat(sep=' ')
  all_words_filtered = all_filtered_text.split(' ')

  all_words_filtered = [text for text in all_words_filtered if len(text) > 0]

  return set(all_words_filtered[:200])

## Tokenização

> Tokenização é o processo de formar palavras de uma sequência de caracteres em um documento    

**Obs: Texto extraido do livro**

E para formar essas palavras podemos usar alguns critérios, como:
- Quantidade de caracteres; Palavras hifenizadas; Caracteres especiais;
- Caracteres em Caixa Alta ou Baixa; Apostrófos; Números; Abreviações;

Cada critério desse, pode guardar uma importância, se tratando dessa base de dados de notícias, alguns poderão ser mais relevantes que outros e veremos o motivo.

Para conseguir concluir quais critérios usar no processo de tokenização, começaremos analisando os dados, garantindo assim que não ignoremos palavras que são importantes.

Se você quiser pular essa parte de análise, e ir para os critérios de tokenização que irei considerar, [clique aqui](https://colab.research.google.com/drive/1EP_hx0xDOgID34aTHwCNL-AFgBWvrGcH?authuser=1#scrollTo=hz-6d5cp6zEJ), mas tenha em mente que só explico os fatores que me levaram a tomar essa decisão aqui embaixo.

### Quantidade de caracteres

A primeira suposição é que palavras com quantidade menores que 3 geralmente não são importantes, porque representam artigos, preposições etc, que não agregam tanto valor na busca.

#### Resultados

In [0]:
get_words_filtered(r'\b\w{0,3}\b')


A título de informação, só mostramos o set das 200 primeiras, mas olhando os dados podemos observar coisas interessantes como:

- Palavras capitalizadas indicam Siglas - APA, PIB, IPI, IVA, PEC, RJ
- Números só com um dígito, não são relevantes
- Ocorrem palavras com tamanho menor ou igual a dois que são importantes, indicam siglas, como 'RJ', 'PEC' e outras que o seu próprio significado tem muito valor, como 'gay' e 'pai'

Sendo assim, removeremos as palavras de tamanho 1 e as stopwords do brasil.


### Palavras hífenizadas

Provavelmente palavras com hífen serão muito importantes, porque elas representam dias da semana, nomes geográficos(Grã-Bretanha), certas espécies de animais e vegetais(Bem-te-vi, Couve-Flor) alguns adjetivos (mal-humorada) dentre outras.

In [0]:
get_words_filtered(r'[a-zA-Z]+\-[a-zA-Z]+')


De fato, foram encontrados dias da semana, partidos, cargos, e muitas palavras importantes com hífen demonstrando que elas não devem ser removidas.

### Caracteres especiais

Também podem ser importantes, basta pensarmos em '%' que pode indicar percentuais, '$' que pode indicar dinheiro, 'º' que pode inficar uma posição, enfim, veremos a seguir, se esses símbolos são importantes no texto.

Olhando símbolos nos textos, além da nossa suposição, foram encontrados certos delimitadores, que não acrescentam informações no texto como:
- '\r\n'
- '\xaO'  
Sendo assim, removeremos eles.

### Apóstrofos

Podem indicar marcas (Burguer’s King, McDonald’s) extrangeiras, ou uma “composição” de palavras como: pingo d’ água

#### Resultados

In [0]:
get_words_filtered(r"\w+\'\w*")

As palavras encontradas com apóstrofo,  provavlmente são erros de digitação/formatação, fazendo mais sentido da replace pra um caractere vazio, e não considerar o hífen na tokenização.

### Números

Números são extremamente importantes, podem indicar horas, salários, anos, posições, temperaturas, etc.

#### Resultados


In [0]:
get_words_filtered(r'\d+\w+')

Confirmando a suposição, não é interessante removê-los.

### Abreviações

Podem agregar muito valor pois servem para indicar cidades, siglas e etc..

### Resultados

In [0]:
get_words_filtered(r'(\w+\.\w+)*')

Assim, como no caso anterior, não é bom que eliminemos, pois temos números decimais, domínios de sites e etc.

## Conlusões dos critérios de tokenização

Da análise anterior, fica claro, que removeremos apóstrofos, palavras de tamanho igual a 1, stopwords e alguns símbolos de escape.

## Criando os tokens



#### Entendendo regex



```regex
\w{2,}[\-*|\.*]\w+|\d{2,}[\.*|\,*]\d+|\w{2,}|\d{2,}'

```
Quebramos em tokens pelos seguintes critérios:
- Palavras com tamanho maior que 2: `\w{2,}`
- Números com quant. de algarismos maior que 2 com símbolos: `\w?\W*\d{2,}\W*`
- Palavras hifenizadas, ou com ponto:`\w{2,}[\-*|\.*]\w+`
- Números decimais com símbolos('$', '%'): `\w?\d{2,}[\.|\,]*\d+\W*`

In [0]:
stopwords = set(stopwords.words('portuguese'))

In [0]:
all_text = clean_text.str.cat(sep=' ')

hyphenated_and_abreviated_words = '\w{2,}[\-|\.]*\w+'
decimal_numbers_with_symbols = '\w?\W?\d{2,}[\.|\,]*\d+\W?'
words ='\w{2,}'
numbers_with_symbols = '\w?\W?\d{2,}\W?'


word_list = regexp_tokenize(all_text, pattern='%s|%s|%s|%s' %(hyphenated_and_abreviated_words, decimal_numbers_with_symbols, words, numbers_with_symbols))
word_list = pd.Series(word_list)

In [12]:
word_list_whithout_stopwords = word_list[~word_list.isin(stopwords)]
word_list_whithout_stopwords = pd.DataFrame(word_list_whithout_stopwords, columns=['word'])
unique_words_whithout_stopwords = word_list_whithout_stopwords['word'].unique()
word_list_whithout_stopwords.head(30)

,word
0,Pessoas
2,buscam
4,identidade
5,transsexual
7,nasceram
8,assim
9,crianças
11,devem
12,ser
13,estimuladas


In [0]:
def get_percentage(value):
  return value * 100

In [14]:
frequencies = word_list_whithout_stopwords['word'].value_counts()
relative_frequencies = word_list_whithout_stopwords['word'].value_counts(normalize=True)
relative_frequencies = relative_frequencies.apply(get_percentage)

relative_frequencies.head(10)

ser           0.447225
disse         0.421750
sobre         0.393444
governo       0.390614
anos          0.382122
Brasil        0.294376
presidente    0.280223
Em            0.280223
pode          0.274562
ainda         0.271731
Name: word, dtype: float64

In [15]:
ranking = np.arange(1,len(unique_words_whithout_stopwords)+1)

most_frequent_words = {
    'frequency': frequencies,
    'r': ranking,
    'Pr(%)': relative_frequencies,
    'r.Pr': relative_frequencies * ranking
}

most_frequent_words = pd.DataFrame(most_frequent_words)
most_frequent_words.head(50)

,frequency,r,Pr(%),r.Pr
ser,158,1,0.447225,0.447225
disse,149,2,0.421750,0.843500
sobre,139,3,0.393444,1.180333
governo,138,4,0.390614,1.562456
anos,135,5,0.382122,1.910612
Brasil,104,6,0.294376,1.766254
presidente,99,7,0.280223,1.961561
Em,99,8,0.280223,2.241784
pode,97,9,0.274562,2.471058
ainda,96,10,0.271731,2.717314


### Stemming

In [0]:
st = RSLPStemmer()
  
def to_stem_tokens(token):
  return [token, st.stem(token)]

In [17]:
stemming_tokens = word_list_whithout_stopwords['word'].apply(to_stem_tokens).tolist()
stemming_tokens = pd.DataFrame(stemming_tokens, columns=['token', 'stem_token']) 
stemming_tokens.head(30)

,token,stem_token
0,Pessoas,pesso
1,buscam,busc
2,identidade,ident
3,transsexual,transsex
4,nasceram,nasc
5,assim,assim
6,crianças,crianç
7,devem,dev
8,ser,ser
9,estimuladas,estimul


In [0]:
tokens_group_by_stems = stemming_tokens.groupby('stem_token')
for name_of_the_group, group in tokens_group_by_stems:
  print(name_of_the_group)
  print(group)

### The Good, the Bad and the Ugly

**Falsos Positivos Encontrados**:

|id | stem_token | tokens  |  amiss  |
| :---: | :---: |  :---:  |   :---:  | 
| 1. | gen | [genial, genes]  |  genial  | 
| 2. | ele | [eleições, eles]  |  eleições  |   
| 3. | far | [farei, faria, fará, farinha]  | farinha |
| 4. | nov | [novas, novo, nova, novamente]  |  novamente  |   
| 5. | natur | [naturais, natureza, naturalizado]  |  naturalizado  |    
| 6. | mão | [mães, mão]  |  mães | 
| 7. | mus | [musas, museu]  |  musas  |
| 8. | mult | [multas, multidões]  |  multidões  |  
| 9. | montante | [montadoras, montadora, montado, montante]  |  montante |
| 10. | leit | [leite, leitura, leituras]  |  leitura  |




**Falsos Negativos Encontrados**:

|id |  stem_token | tokens  |  wrong_stem_token  |    wrong_token  |
| :---: | :---: |  :---:  |   :---:  |     :---:  |
| 1. | aproveit | [aproveita, aproveitamento, aproveitaram]  |  aprove  |    aproveitar  |
| 2. | aposentad | [aposentadoria]  |  aposent  |    aposentado  |
| 3. | arrecad | [arrecadados, arrecadação, arrecadado, arrecadar]  |  arrec  |    arrecada  |
| 4. | assegur | [assegurar]  |  asseg  |    assegura  |
| 5. | assessor | [assessora, assessor, assessores]  |  assess  |    assessoria  |
| 6. | deliber | [deliberar, deliberações]  |  deliberá-l  |    deliberá-lo  |
| 7. | demand | [demanda]  |  demandar-s  |    demandar-se  |
| 8. | emocion | [emocionaram]  |  emoc  |   emocional  |
| 9. | habilit | [habilitado]  |  habil |    habilitar |
| 10. | orçament | [orçamentário]  |  orç |    orçamento  |






